In [1]:
import os
import re
import numpy as np

from pororo import Pororo
from jamo import h2j, j2hcj

from io import StringIO
from contextlib import redirect_stdout
from koparadigm import Paradigm, prettify

from pkg.util import read_problemsheet, write_problemsheet
from pkg.parse import *
from pkg.words import *
from pkg.dataset import ProblemDataset, read_questions, write_questions

from problems import *

import torch, torch.nn as nn, torch.nn.functional as F
import time

import sentencepiece as spm

num_epochs = 100
num_files = 10

In [2]:
from pororo import Pororo
gec, gec_factory = Pororo(task="gec", lang="kr")

Pororo Factory Base, task_config TaskConfig(task='gec', lang='ko', n_model='charbert.base.ko.spacing')
Init PororoGecFactory
gec
ko
None
As of now, this beta model tries to correct spacing errors in Korean text.


In [3]:
dir_data = 'data/question'
if not os.path.exists(dir_data):
    os.mkdir(dir_data)
    
path_question = os.path.join(dir_data, 'question.txt')

In [4]:
problems = [P1_1_1, P1_1_2, P1_1_3, P1_1_4, P1_1_5, P1_1_6, P1_1_7, P1_1_8, P1_1_9, P1_1_10, P1_1_11, P1_1_12, 
            P1_2_1, P1_2_2, P1_3_1, P1_4_1, 
            P2_1_1, P2_2_2, P2_3_1, 
            P3_1_1, P3_2_1, P3_2_2, P3_3_1, 
            P4_1_1, P4_2_1, P4_2_2, P4_3_1, 
            P5_1_1, P5_2_1, P5_3_1,
            P6_1_1, P6_3_1, P6_4_1,
            P7_1_1, P7_1_2, P7_3_1,
            P8_1_1, P8_2_1, P8_3_1, 
            P9_1_1, P9_2_1, P9_2_2, P9_3_1, P9_3_2]

In [5]:
ds = ProblemDataset(problems, batch_size=64, gec=gec)

In [6]:
def create_dataset(ds, num_epochs=100, path_meta=None):
    start = time.time()
    count = 0
    meta = []
    for i in range(num_epochs):
        if i % 10 == 0:
            print(i, count, 'elapsed:', time.time() - start)
        for j in range(len(ds)):
            batch = ds[j]
            idx = batch['idx']
            pname = batch['pname']
            text = batch['text']
            for k in range(len(idx)):
                meta.append([idx[k], pname[k], text[k]])
            count += len(batch['text'])
    print(time.time() - start)
    print('num of lines', count)
    if path_meta is not None:
        write_questions(meta, path_meta)


In [ ]:
for i in range(num_files):
    print('# ===================================', str(i), 'th file. ===================================')
    path_meta = os.path.join(dir_data, 'question_' + str(i) + '.txt')
    create_dataset(ds, num_epochs=num_epochs, path_meta=path_meta)

# =================================== 0 th file. ===================================
0 0 elapsed: 4.291534423828125e-06
10 28160 elapsed: 86.09882283210754
20 56320 elapsed: 176.0097496509552
30 84480 elapsed: 265.31213331222534
40 112640 elapsed: 355.0499289035797
50 140800 elapsed: 445.144202709198
60 168960 elapsed: 535.1779870986938
70 197120 elapsed: 624.9152910709381
80 225280 elapsed: 714.0475697517395
90 253440 elapsed: 803.9934153556824
893.8961129188538
num of lines 281600
# =================================== 1 th file. ===================================
0 0 elapsed: 3.337860107421875e-06
10 28160 elapsed: 89.39284157752991
20 56320 elapsed: 179.51828503608704
30 84480 elapsed: 269.1716947555542
40 112640 elapsed: 358.7031376361847
50 140800 elapsed: 448.6218378543854
60 168960 elapsed: 538.6979522705078
70 197120 elapsed: 628.216653585434
80 225280 elapsed: 717.7226831912994
90 253440 elapsed: 807.6290240287781
897.1301054954529
num of lines 281600
# ======================